In [ ]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
data_dir = "/Users/pervin0527/upstage-ml-regression-ml11/data/new"
train_data = pd.read_csv(f"{data_dir}/new_train.csv")
test_data = pd.read_csv(f"{data_dir}/new_test.csv")

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
train_data['is_test'] = 0
test_data['is_test'] = 1

data = pd.concat([train_data, test_data])
data['is_test'].value_counts() 

In [ ]:
missing_data = data[data[['좌표X', '좌표Y']].isnull().any(axis=1)]
print(len(missing_data))

In [ ]:
values = []
for i, (index, row) in enumerate(missing_data.iterrows()):
    city = row['시군구']
    road_name = row['도로명']

    value = ' '.join(city.split(' ')[:-1]) + " " + road_name
    values.append(value)

print(len(values))

values_set = set(values)
print(values_set)

In [ ]:
url = "https://www.findlatlng.org/"
browser = webdriver.Chrome()
browser.get(url)

results = {}
for idx, value in enumerate(values_set):
    print(f"{idx:>08}, 검색어: {value}")

    search_element = browser.find_element(By.CLASS_NAME, "form-control")
    if idx > 0:
        search_element.clear()
    
    search_element.send_keys(value)
    search_element.send_keys(Keys.RETURN)
    time.sleep(3)

    address_div = browser.find_element(By.XPATH, '//*[@id="app"]/div/div[3]')
    text_content = address_div.text
    
    lines = text_content.split('\n')
    for line in lines:
        if "위도(Latitude)" in line:
            latitude = line.split(': ')[1].split(' /')[0].strip()
        if "경도(Longitude)" in line:
            longitude = line.split(': ')[2].strip()
    
    # 결과를 딕셔너리에 저장
    results[value] = {'Y좌표': latitude, 'X좌표': longitude}
    print(f"X좌표: {longitude}, Y좌표: {latitude}\n")

In [ ]:
for i, (index, row) in enumerate(missing_data.iterrows()):
    city = row['시군구']
    road_name = row['도로명']
    value = ' '.join(city.split(' ')[:-1]) + " " + road_name

    if value in results:
        y, x = results[value]
        missing_data.loc[index, 'X좌표'] = float(x)
        missing_data.loc[index, 'Y좌표'] = float(y)

data.update(missing_data)

train_df = data[data['is_test'] == 0].drop(columns=['is_test'])
test_df = data[data['is_test'] == 1].drop(columns=['is_test'])

os.makedirs(f'{data_dir}/new', exist_ok=True)
train_df.to_csv(f'{data_dir}/new/new_train.csv', index=False)
test_df.to_csv(f'{data_dir}/new/new_test.csv', index=False)

browser.quit()